<a href="https://colab.research.google.com/github/wrightchin/vgg16_transfer_learning/blob/main/vgg16_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to ~/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

In [ ]:
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [ ]:
# Load vgg16 model  

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
# Last Layer

In [ ]:
from tensorflow.keras import layers, models

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

In [ ]:
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
# Fit the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

In [ ]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 17s 120ms/step - loss: 1.7802 - accuracy: 0.3212 - val_loss: 1.4488 - val_accuracy: 0.4903
Epoch 2/50
65/65 [==============================] - 6s 96ms/step - loss: 1.2573 - accuracy: 0.5445 - val_loss: 1.2365 - val_accuracy: 0.5156
Epoch 3/50
65/65 [==============================] - 6s 95ms/step - loss: 0.9989 - accuracy: 0.6550 - val_loss: 1.0303 - val_accuracy: 0.6479
Epoch 4/50
65/65 [==============================] - 6s 94ms/step - loss: 0.7507 - accuracy: 0.7421 - val_loss: 1.1469 - val_accuracy: 0.6459
Epoch 5/50
65/65 [==============================] - 6s 95ms/step - loss: 0.5545 - accuracy: 0.7883 - val_loss: 1.1288 - val_accuracy: 0.6518
Epoch 6/50
65/65 [==============================] - 6s 97ms/step - loss: 0.4567 - accuracy: 0.8151 - val_loss: 1.1320 - val_accuracy: 0.6868
Epoch 7/50
65/65 [==============================] - 6s 95ms/step - loss: 0.3811 - accuracy: 0.8540 - val_loss: 1.0087 - val_accuracy: 0.6732
Epoch 8/50


In [ ]:
model.evaluate(test_ds, test_labels)

35/35 [==============================] - 3s 76ms/step - loss: 0.1960 - accuracy: 0.9246


[0.19602251052856445, 0.9246140122413635]